In [21]:
from mongodb_lib import *
import yaml
import pandas as pd
from collections import defaultdict
from openai_handlers import *
import re
import ast

In [22]:
config_infra = yaml.load(open("../infra-config-pipeline.yaml"), Loader=yaml.FullLoader)
db, fs, client = connect_to_mongodb(config_infra)

INFO:root:Successfully connected to MongoDB.


In [71]:
# Import taxonomy

taxonomy = pd.read_excel("../Categories.xlsx")
taxonomy = taxonomy[["Sub-category", "Description & Keywords.1"]]

taxonomy_json = defaultdict(lambda: defaultdict(list))

list_sub_categories = list()
description2id = {}

for key, value in zip(taxonomy["Sub-category"], taxonomy["Description & Keywords.1"]):

    entries = value.split("\n")
    assert len(entries) == 2

    list_sub_categories.append(entries[0].replace("- Description: ",""))

    print(entries[0].replace("- Description: ",""))

Museums dedicated to the collection and exhibition of visual arts.
Museums focused on historical artifacts and exhibits that tell the story of past events and cultures.
Museums that focus on classical antiquities and ancient civilizations.
Interactive museums that offer immersive experiences and hands-on exhibits.
Guided walking tours through cities, highlighting historical landmarks and cultural sites.
Tours of historical battlefields, providing insights into military history and significant battles.
Tours that focus on the historical and cultural heritage of a region.
Tours that explore archaeological sites and ancient ruins.
Guided tours of historical castles and palaces.
Festivals that focus on live music performances across various genres.
General cultural festivals that celebrate various aspects of culture and heritage.
Live theatrical performances, including plays, musicals, and shows.
Live music performances, including concerts, operas, and musical theater.
General live perform

In [66]:
taxonomy_json

defaultdict(<function __main__.<lambda>()>,
            {'1.1.1: Art Museums': defaultdict(list,
                         {'description': 'Museums dedicated to the collection and exhibition of visual arts.',
                          'keywords': 'Paintings, sculptures, contemporary art, classical art, exhibitions.'}),
             '1.1.2: History Museums': defaultdict(list,
                         {'description': 'Museums focused on historical artifacts and exhibits that tell the story of past events and cultures.',
                          'keywords': 'Historical artifacts, ancient civilizations, historical events, cultural heritage.'}),
             '1.1.3: Classical Museum': defaultdict(list,
                         {'description': 'Museums that focus on classical antiquities and ancient civilizations.',
                          'keywords': 'Ancient Greece, Rome, classical artifacts, antiquities.'}),
             '1.1.4: Experience Museum': defaultdict(list,
                    

In [24]:
data = read_object(fs, "product_textual_lang_summarized")
data = pd.DataFrame(data)
data.fillna("", inplace=True)
data = data[["pdt_product_detail_PRODUCTDESCRIPTION_SUMMARIZED"]]
data.columns = ["description"]
data = data.sample(frac=1).reset_index(drop=True)

In [25]:
apikey="sk-proj-CXQXeS2YM9i9oNBJvKvcT3BlbkFJ9nzPvAG7HhK9YRGVco6A"

In [31]:
conversation_history = [
    {"role": "system", "content": "Hello! How can I assist you today?"}
]

prompt = (
    "You are a multi-label classifier tasked with categorizing texts into one or more categories. "
    "In the next prompts, I will provide you with a Python list of texts, and you should return a Python list of lists "
    "with applicable labels for each text. If no labels apply to a text, return an empty list ([]).\n\n"
    "Example: If a text relates to history and archaeology, return ['1.1.2: History Museums', '1.2.4: Archaeology Tours'] for that text.\n\n"
    "Return a Python list where each element corresponds to the labels for the texts provided, in respective order.\n\n"
    "Provide your response as a list of lists.\n\n"
    "Example input format: [['Text 1'], ['Text 2'], ['Text 3'], ['Text 4']]\n\n"
    "Example output format: [['1.1.2: History Museums'], ['1.2.4: Archaeology Tours'], [], ['2.5.2: Wine Festivals', '2.1.2: Market Tours']]\n\n"
    ""
    f"Here are the possible labels:\n{taxonomy}\n\n"
    "Are you ready to begin?"
)

result = query_gpt_with_history(apikey, prompt, conversation_history)
result = result.choices[0].message.content
conversation_history.append({"role": "user", "content": prompt})
conversation_history.append({"role": "system", "content": result})
print(result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Yes, I'm ready! Please provide me with the Python list of texts that you'd like me to classify.


In [62]:
texts = [data['description'].tolist()[0]]
result = query_gpt_with_history(apikey, str(texts), conversation_history)
categories_string = re.findall(r"\[\[.*?\]\]", result.choices[0].message.content, re.DOTALL)[0]
result = ast.literal_eval(categories_string)

if len(result) != len(texts):
    
    print(f"Warning: Expected {len(texts)} results but got {len(result)}.")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [51]:
result

[['1.1.2: History Museums', '1.6.1: Literary, Art & Music Tours'],
 ['1.2.1: City Walking Tours']]

In [52]:
texts

['130 works to view. different countries: England, France, the United States and Palestine. 3 floors to see containing different countries.    130 works to see. Different countries: France, England, the U.S., Palestine.',
 '2-hour tour of the city by coach, with live commentary from an English-speaking guide. Some sights include Place de l’Opéra, the Louvre Museum, the Eiffel Tower, Notre Dame Cathedral, the Arc de Triomphe and more.']

In [63]:
result

[['1.1.2: History Museums', '1.1.3: Classical Museum', '1.4.3: Performances']]

In [64]:
texts

['130 works to view. different countries: England, France, the United States and Palestine. 3 floors to see containing different countries.    130 works to see. Different countries: France, England, the U.S., Palestine.']